In [1]:
#################### import all libraries and initializations ############

import sys
import numpy as np 
import os
import time
import math
from PIL import Image
import matplotlib.pylab as plt
import cv2
from datetime import datetime
from pynq import Xlnk
from pynq import Overlay
from preprocessing import *
import struct
from pynq import MMIO
import pynq

team = 'BUET_X'
agent = Agent(team)
interval_time = 0

xlnk = Xlnk()
xlnk.xlnk_reset()

img = xlnk.cma_array(shape=(3,178,322), dtype=np.uint8)

conv_weight_1x1_all = xlnk.cma_array(shape=(186, 16, 16), dtype=np.uint16)
conv_weight_3x3_all = xlnk.cma_array(shape=(7, 16, 3, 3), dtype=np.uint16) 
bias_all = xlnk.cma_array(shape=(16), dtype=np.uint16)

DDR_pool_3_out = xlnk.cma_array(shape=(32, 90, 162), dtype=np.uint16)
DDR_pool_6_out = xlnk.cma_array(shape=(64, 46, 82), dtype=np.uint16)
DDR_buf = xlnk.cma_array(shape=(36, 16, 24, 42), dtype=np.uint16)

predict_box = xlnk.cma_array(shape=(5,), dtype=np.float32)

print("Allocating memory done")

img_path = '/home/xilinx/jupyter_notebooks/dac_2019_contest/images/'
coord_path = '/home/xilinx/jupyter_notebooks/dac_2019_contest/result/coordinate/BUET_X/'

tbatch = 0
total_num_img = len(agent.img_list)
result = list()
agent.reset_batch_count()        
        
blank = Image.new('RGB', (322, 178), (127, 127, 127))

# load parameters from SD card to DDR

params = np.fromfile('/home/xilinx/jupyter_notebooks/dac_2019_contest/BUET_X/params/BUET_X.bin', dtype=np.uint16)

idx = 0

np.copyto(conv_weight_1x1_all, params[idx:idx+conv_weight_1x1_all.size].reshape(conv_weight_1x1_all.shape))
idx += conv_weight_1x1_all.size

np.copyto(conv_weight_3x3_all, params[idx:idx+conv_weight_3x3_all.size].reshape(conv_weight_3x3_all.shape))
idx += conv_weight_3x3_all.size

np.copyto(bias_all, params[idx:idx+bias_all.size].reshape(bias_all.shape))

print("Parameters loading done")


Allocating memory done
Parameters loading done


In [7]:
################### download the overlay #####################
overlay = Overlay('/home/xilinx/jupyter_notebooks/dac_2019_contest/BUET_X/overlay/BUET_X.bit')
print("BUET_X.bit loaded")
myIP = overlay.model_shift_0

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:299: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


model_shift.bit loaded


In [8]:
################## download weights and image resizing and processing
myIP.write(0x10, img.physical_address)

myIP.write(0x18, conv_weight_1x1_all.physical_address)
myIP.write(0x20, conv_weight_3x3_all.physical_address)
myIP.write(0x28, bias_all.physical_address)

myIP.write(0x30, DDR_pool_3_out.physical_address)
myIP.write(0x38, DDR_pool_6_out.physical_address)

myIP.write(0x40, DDR_buf.physical_address)
myIP.write(0x48, predict_box.physical_address)


interval_time = 0
total_time = 0
total_num_img = len(agent.img_list)
result = list()
agent.reset_batch_count()
rails = pynq.get_rails()

first_image = True
boxes = []
names = []


recorder = pynq.DataRecorder(rails['power1'].power)
with recorder.record(0.01):  
    for i in range(math.ceil(total_num_img/BATCH_SIZE)):
        # get a batch from agent
        batch = agent.send(interval_time, agent.img_batch)

        for currPic in batch:
            names.append(currPic)
            # timer start when PS reading image
            start = time.time()
            if first_image:
                    image = Image.open(img_path + currPic).convert('RGB')
                    image = image.resize((320, 176))
                    blank.paste(image, (1, 1))
                    image = np.transpose(blank, (2, 0, 1))
                    np.copyto(img, np.array(image))

                    first_image = False
                    continue

            if not first_image:
                myIP.write(0x00, 1)
                time.sleep(0.07)
                image = Image.open(img_path + currPic).convert('RGB')
                image = image.resize((320, 176))
                blank.paste(image, (1, 1))
                image = np.transpose(blank, (2, 0, 1))
                np.copyto(img, np.array(image))

            isready = myIP.read(0x00)
            while( isready == 1 ):
                isready = myIP.read(0x00)

            predict_box[0] = predict_box[0] / 40;
            predict_box[1] = predict_box[1] / 22;
            predict_box[2] = predict_box[2] / 40;
            predict_box[3] = predict_box[3] / 22;
            #print(predict_box)
            x1 = int(round((predict_box[0] - predict_box[2]/2.0) * 640))
            y1 = int(round((predict_box[1] - predict_box[3]/2.0) * 360))
            x2 = int(round((predict_box[0] + predict_box[2]/2.0) * 640))
            y2 = int(round((predict_box[1] + predict_box[3]/2.0) * 360))
            boxes.append([x1, x2, y1, y2])
            print([x1, x2, y1, y2])
            end = time.time()
            t = end - start
            total_time += t


    start = time.time()        
    #collect result for last image
    myIP.write(0x00, 1)
    isready = myIP.read(0x00)
    while( isready == 1 ):
        isready = myIP.read(0x00)   

    predict_box[0] = predict_box[0] / 40;
    predict_box[1] = predict_box[1] / 22;
    predict_box[2] = predict_box[2] / 40;
    predict_box[3] = predict_box[3] / 22;

    x1 = int(round((predict_box[0] - predict_box[2]/2.0) * 640))
    y1 = int(round((predict_box[1] - predict_box[3]/2.0) * 360))
    x2 = int(round((predict_box[0] + predict_box[2]/2.0) * 640))
    y2 = int(round((predict_box[1] + predict_box[3]/2.0) * 360))
    boxes.append([x1, x2, y1, y2])
    print([x1, x2, y1, y2])
    
# timer stop after PS has processed all the batches
    end = time.time()
    t = end - start
    total_time += t

energy = recorder.frame["power1_power"].mean() * total_time

print('Processing time: {} seconds.'.format(total_time))
print('Energy: {} J.'.format(energy))

        
print("All computation done")

[312, 380, 261, 322]
[399, 407, 247, 264]
[310, 335, 202, 243]
[272, 317, 100, 178]
Processing time: 0.47536563873291016 seconds.
Energy: 2.2681731905255997 J.
All computation done


In [5]:
################ record the results and write to XML
f_out = open(coord_path + '/BUET_X.txt', 'w')

cnt = 0
for box in boxes:
    x1 = box[0]
    x2 = box[1]
    y1 = box[2]
    y2 = box[3]
    coord = str(x1) + ' ' + str(x2) + ' ' + str(y1) + ' ' + str(y2)
    
    name = names[cnt]
    cnt = cnt + 1
    f_out.write(name + '\n')
    f_out.write(coord + '\n')
        
f_out.close()
print("\nAll results stored in BUET_X.txt")

agent.save_results_xml(boxes,total_time, energy)
print("XML results written successfully.")



All results stored in BUET_X.txt
XML results written successfully.


In [29]:
xlnk.xlnk_reset()